In [1]:
import requests 
import json 
import pandas as pd
import time
import urllib
import urllib.request 
from urllib.error import HTTPError
from bs4 import *
import os
#!pip install unidecode
import unidecode

from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
import cv2
from PIL import Image, UnidentifiedImageError
from pdb import set_trace
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv 
import glob
import dlib
import numpy as np
import random
from requests.adapters import HTTPAdapter
import torch
import hashlib
import shutil
import tempfile
from urllib.request import urlopen, Request

!pip install facenet_pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face

use_cuda=torch.cuda.is_available()
device=torch.device("cuda" if use_cuda else "cpu")
print("Using %s device"%(device))

################3
#Global variables
###################
#Change the following path to the folder containing the images
data_dir=r"C:\Users\mihnea.andrei\Work\facial_recognition\images"
save_dir=r"C:\Users\mihnea.andrei\Work\facial_recognition\images"
nr_repeats=11
perc_eval=0.2
nr_train=nr_repeats-int(perc_eval*nr_repeats)
horiz_flip=transforms.RandomHorizontalFlip(p=0.5)
color_jitter=transforms.ColorJitter(brightness=0.5,contrast=0.3)
gaussian_blur=transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
adj_sharpness=transforms.RandomAdjustSharpness(sharpness_factor=2)
picture_formats=["jpg","jpeg","png","gif", "pdf", "png", "tiff"]
printing_iter=10**(2)
detector=MTCNN()


def timming(since):
    now=time.time() 
    s=now-since
    m=int(s/60)
    s=s-m*60
    return "%dm %ds"%(m,s)


count_links=13800
count_people=0
count_faces=0
start=time.time()
file1=open(r"C:\Users\mihnea.andrei\Work\facial_recognition\wikipedia_face_parser\Print_details.txt","w")
print_data=""
image_name=""

while True:
    prefix="Q"+str(count_links)
    flag=1
    wiki_link="https://www.wikidata.org/wiki/%s"%(prefix)
    json_link="https://www.wikidata.org/wiki/Special:EntityData/%s.json"%(prefix) 
    count_links+=1
    try:
        response=requests.get(json_link)
    except requests.exceptions.Timeout:
        print(100*"-")
        print("Too much time to request wikipedia link %s"%(image_link))
        print(100*"-")
        flag=0
    except requests.exceptions.TooManyRedirects:
        print(100*"-")
        print("Bad wikipedia link %s"%(image_link))
        print(100*"-")
        flag=0
    except requests.exceptions.RequestException as e:
        print(100*"-")
        print("Catastrophic error for wikipaedia link %s"%(image_link))
        print(100*"-")
        flag=0
    if flag==1:
        try: 
            person=json.loads(response.content)
        except ValueError as err:
            flag=0
    if flag==1:
        if prefix in person["entities"].keys(): 
            if "P31" in person["entities"][prefix]["claims"].keys():
                if "mainsnak" in person["entities"][prefix]["claims"]["P31"][0].keys():
                    if "datavalue" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"].keys():
                        if "value" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"].keys():
                            if "id" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"].keys():
                                if person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"]["id"]=="Q5":
                                    #############3
                                    #Obtain the name
                                    ##############3
                                    person_name=""
                                    if "entities" in person.keys():   
                                        if prefix in person["entities"].keys():
                                            if "labels" in person["entities"][prefix].keys(): 
                                                if "en" in person["entities"][prefix]["labels"].keys():
                                                    person_name=person["entities"][prefix]["labels"]["en"]["value"]
                                                    #remove quotes since it gives an error when the image file name when saved
                                                    if '"' in person_name:
                                                        person_name=person_name.replace('"',"")
                                                    if "/" in person_name:
                                                        person_name=person_name.replace('/',"")
                                                    if " "in person_name:
                                                        person_name=person_name.replace(' ',"")
                                                    if "\\" in person_name:
                                                        person_name=person_name.replace(' ',"")
                                                    if "." in person_name:
                                                        person_name=person_name.replace('.',"")
                                                    if "?" in person_name:
                                                        person_name=person_name.replace("?","")
                                                    if "!" in person_name:
                                                        person_name=person_name.replace("!","")
                                                    #remove accents, diacritics
                                                    person_name=unidecode.unidecode(person_name)
                                    ##########
                                    #Obtain the image
                                    #########
                                    if person_name!="":
                                        if "claims" in person["entities"][prefix].keys():
                                            if "P18" in person["entities"][prefix]["claims"].keys():
                                                if "mainsnak" in person["entities"][prefix]["claims"]["P18"][0].keys():
                                                    if "datavalue" in person["entities"][prefix]["claims"]["P18"][0]["mainsnak"].keys():
                                                        if "value" in person["entities"][prefix]["claims"]["P18"][0]["mainsnak"]["datavalue"].keys():
                                                            image_name=person["entities"][prefix]["claims"]["P18"][0]["mainsnak"]["datavalue"]["value"]
                                                            if '"' in image_name:
                                                                image_name=image_name.replace('"','')
                                                            if "\\" in image_name:
                                                                image_name=image_name.replace("\\","")
                                                            image_suffix="_".join(image_name.split(" "))
                                                            image_link="https://www.wikidata.org/wiki/%s#/media/File:%s"%(prefix,image_suffix)
                                                            #image_link = urllib.parse.quote(link,safe=':/')
                                                            try:
                                                                r=requests.get(image_link)
                                                            except requests.exceptions.Timeout:
                                                                print("Too much time to request image link %s"%(image_link))
                                                                flag=0
                                                            except requests.exceptions.TooManyRedirects:
                                                                print("Bad image link %s"%(image_link))
                                                                flag=0
                                                            except requests.exceptions.RequestException as e:
                                                                print("Catastrophic error for image link %s"%(image_link))
                                                                flag=0
                                                            if flag==1:
                                                                #parse HTML code
                                                                soup=BeautifulSoup(r.text,"html.parser")
                                                                #find all images in url (should only be 1 here)
                                                                images=soup.findAll('img')
                                                                if len(images)!=0:
                                                                    image=images[0]
                                                                    try:
                                                                        image_link=image["src"]
                                                                    except:
                                                                        flag=0
                                                                    if flag==1:
                                                                        image_link="https:"+image_link
                                                                        file_path=save_dir+"/"+person_name+".jpg"
                                                                        try:
                                                                            urllib.request.urlretrieve(image_link,file_path)
                                                                        except Exception as e:
                                                                            print("%s for image has error %s, link:%s"%(image_name,e,image_link))
                                                                            flag=0
                                                                        if flag==1:
                                                                            save_to_path=r"C:\Users\mihnea.andrei\Work\facial_recognition\faces_images\train"
                                                                            count_people+=1
                                                                            #Need a folder for each person with pictures for that person (in this case 1 like we have for all romanians)
                                                                            #This is because the ImageLOader for resnet reads in the data this way.
                                                                            for i in range(1,nr_repeats+1):
                                                                                try:
                                                                                    image=Image.open(file_path)
                                                                                except (ValueError,UnidentifiedImageError,OSError):
                                                                                    flag=0
                                                                                if flag==1:
                                                                                    save_to_suffix="/".join(file_path.split("/")[-1:]).split(".jpg")[0]+"/"+str(i)+".jpg"
                                                                                    u=random.random()
                                                                                    if 0<=u<0.25:
                                                                                        try:
                                                                                            image=horiz_flip(image)
                                                                                        #except (ValueError, RuntimeError):
                                                                                        except:
                                                                                            pass
                                                                                    if 0.25<=u<0.5:
                                                                                        try:
                                                                                            image=color_jitter(image)
                                                                                        except:
                                                                                            pass
                                                                                    if 0.5<=u<0.75:
                                                                                        try:
                                                                                            image=gaussian_blur(image)
                                                                                        except:
                                                                                            pass
                                                                                    if 0.75<=u<1:
                                                                                        try:
                                                                                            image=adj_sharpness(image)
                                                                                        except:
                                                                                            pass
                                                                                    try:
                                                                                        results=detector.detect(image)
                                                                                    except:
                                                                                        flag=0
                                                                                    if flag==1:
                                                                                        try:
                                                                                            box=results[0][0]
                                                                                        except TypeError:
                                                                                            flag=0
                                                                                        if flag==1:        
                                                                                            for pic_type in picture_formats:
                                                                                                if pic_type in file_path:
                                                                                                    save_to_path+="/"+save_to_suffix.split("."+pic_type)[0]+".jpg"
                                                                                                    break
                                                                                                else:
                                                                                                    print("Picture format not accepted for %s"%(file_path))
                                                                                                    flag=0
                                                                                            if flag==1:
                                                                                                extract_face(image,box,save_path=save_to_path)
                                                                                                if i<=nr_train:
                                                                                                    save_to_path=r"C:\Users\mihnea.andrei\Work\facial_recognition\faces_images\train"
                                                                                                else:
                                                                                                    save_to_path=r"C:\Users\mihnea.andrei\Work\facial_recognition\faces_images\eval"
                                                        
                                                                                                count_faces+=1
    if count_links%printing_iter==0:
        try:
            print("Downloaded %d people \t Tried %d links \t Created %d faces \t Last person: %s \t %s time"%(count_people,count_links,count_faces,person_name,timming(start)))
        except NameError:
            pass
    

Using cpu device
Downloaded 14 people 	 Tried 8800 links 	 Created 67 faces 	 Last person: AemiliusMacer 	 1m 15s time
Downloaded 24 people 	 Tried 8900 links 	 Created 144 faces 	 Last person: JoeyMoe 	 2m 21s time
Downloaded 33 people 	 Tried 9000 links 	 Created 198 faces 	 Last person: MoYan 	 3m 21s time
Downloaded 52 people 	 Tried 9100 links 	 Created 321 faces 	 Last person: TabareVazquez 	 4m 45s time
Downloaded 73 people 	 Tried 9200 links 	 Created 494 faces 	 Last person: ReneDescartes 	 6m 8s time
Downloaded 86 people 	 Tried 9300 links 	 Created 584 faces 	 Last person: ManduulKhan 	 7m 18s time
Downloaded 107 people 	 Tried 9400 links 	 Created 727 faces 	 Last person: SilviusMagnago 	 8m 38s time
Downloaded 120 people 	 Tried 9500 links 	 Created 811 faces 	 Last person: SachinTendulkar 	 9m 50s time
Downloaded 138 people 	 Tried 9600 links 	 Created 976 faces 	 Last person: SusanneAlbers 	 11m 10s time
Downloaded 149 people 	 Tried 9700 links 	 Created 1097 faces 	 Las

KeyboardInterrupt: 